# Improving GME gradient computation efficiency

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import time

import autograd.numpy as npa
from autograd import grad, value_and_grad

import legume
from legume.minimize import Minimize

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## PhC cavity simulation

We will look at the exact same optimization as in Notebook 6 and see how it can be improved both in terms of accuracy and in terms of computational time and memory. We first define the cavity that we will simulate.

In [40]:
# Number of PhC periods in x and y directions
Nx, Ny = 16, 10

# Regular PhC parameters
ra = 0.234
dslab = 0.4355
n_slab = 2.21

# Initialize a lattice and PhC
lattice = legume.Lattice([Nx, 0], [0, Ny*np.sqrt(3)/2])

# Make x and y positions in one quadrant of the supercell
# We only initialize one quadrant because we want to shift the holes symmetrically
xp, yp = [], []
nx, ny = Nx//2 + 1, Ny//2 + 1
for iy in range(ny):
    for ix in range(nx):
        xp.append(ix + (iy%2)*0.5)
        yp.append(iy*np.sqrt(3)/2)

# Move the first two holes to create the L4/3 defect
xp[0] = 2/5
xp[1] = 6/5 
nc = len(xp)

# Initialize shift parameters to zeros
dx, dy = np.zeros((nc,)), np.zeros((nc,))

In [41]:
# Define L4/3 PhC cavity with shifted holes
def cavity(dx, dy):
    # Initialize PhC
    phc = legume.PhotCryst(lattice)
    
    # Add a layer to the PhC 
    phc.add_layer(d=dslab, eps_b=n_slab**2)
    
    # Apply holes symmetrically in the four quadrants
    for ic, x in enumerate(xp):
        yc = yp[ic] if yp[ic] == 0 else yp[ic] + dy[ic]
        xc = x if x == 0 else xp[ic] + dx[ic]
        phc.add_shape(legume.Circle(x_cent=xc, y_cent=yc, r=ra))
        if nx-0.6 > xp[ic] > 0 and (ny-1.1)*np.sqrt(3)/2 > yp[ic] > 0:
            phc.add_shape(legume.Circle(x_cent=-xc, y_cent=-yc, r=ra))
        if nx-1.6 > xp[ic] > 0:
            phc.add_shape(legume.Circle(x_cent=-xc, y_cent=yc, r=ra))
        if (ny-1.1)*np.sqrt(3)/2 > yp[ic] > 0 and nx-1.1 > xp[ic]:
            phc.add_shape(legume.Circle(x_cent=xc, y_cent=-yc, r=ra))
            
    return phc

In Notebook 6, we used a relatively fast optimization for illustration purposes. In the `legume` paper, we use `gmax = 2.5`, as well as a 3x3 $k$-grid in the Brillouin zone, and average the loss rates. This improves the accuracy of the result, especially when going to ultra-high-$Q$ values. However, it also takes a longer time to compute, and significantly more memory if done directly, because all the variables, including all the dense matrices and all the eigenvectors at every $k$ are stored for the backpropagation. Below we will see how we can introduce some improvements.

Below, we define a new GME computation function, with two main differences from Notebook 6. Firstly, we compute the loss rates on a grid of $k$-points. Second, instead of taking the `Nx*Ny` eigenmode as the cavity mode, we look for the first eigenfrequency above 0.41 (first mode in the band gap). The reason for this will become clear below.

In [46]:
def gme_cavity_k(params, gmax, options, kpoints, f_lb=0.41):
    dx = params[0:nc]
    dy = params[nc:]
    phc = cavity(dx, dy)
    options['compute_im'] = False
    gme = legume.GuidedModeExp(phc, gmax=gmax)
    gme.run(kpoints=kpoints, **options)
    indmode = npa.nonzero(gme.freqs[0, :] > f_lb)[0][0]
    fims = []
    for ik in range(kpoints[0, :].size):
        (freq_im, _, _) = gme.compute_rad(ik, [indmode])
        fims.append(freq_im)
    return (gme, npa.array(fims), indmode)

## Accuracy of forward computation
We can now compute the averaged $Q$ in a straightforward way. Note that it's better to average the loss rates and only then compute the $Q$, rather than average the $Q$-s. 

In [49]:
pstart = np.zeros((2*nc, ))

def Q_kavg(params):
    (gme, fims, indmode) = gme_cavity_k(params, gmax, options, np.vstack((kxg, kyg)))
    print(gme.freqs[:, indmode], fims)
    # Note that the real part of the freq doesn't change much so we can just take ik=0 below
    return gme.freqs[0, indmode]/2/npa.mean(fims)

# First compute the Q at k = [0, 0] like we did in Notebook 6
kxg = np.array([0])
kyg = np.array([0])
print("Quality factor computed using k=[0,0]: %1.2f" %Q_kavg(pstart))

# Create an nkx x nky grid in k space (positive kx, ky only)
nkx = 3
nky = 3
kx = np.linspace(0, np.pi/Nx, nkx)
ky = np.linspace(0, np.pi/Ny/np.sqrt(3)*2, nky)
kxg, kyg = np.meshgrid(kx, ky)
kxg = kxg.ravel()
kyg = kyg.ravel()

print("Quality factor averaged over %d k-points: %1.2f" %(nkx*nky, Q_kavg(pstart)))

[0.41872192] [[3.24380871e-05]]
Quality factor computed using k=[0,0]: 6454.17
[0.41872192 0.42720883 0.42775149 0.41854415 0.4277026  0.4292372
 0.43178911 0.43219462 0.41866084] [[3.24380871e-05]
 [2.42209237e-06]
 [1.07073795e-06]
 [3.18673397e-05]
 [1.82460426e-06]
 [1.25038284e-06]
 [9.24534102e-07]
 [1.06510447e-06]
 [3.50534814e-05]]
Quality factor averaged over 9 k-points: 17460.27


This $Q$-value is three times smaller than what we got above, and is likely closer to the true quality factor. For a converged optimization, it's typically good to average over a small grid in $k$-space as shown here. To use this in an optimization, we can already use `Q_kavg` as an objective function, but, like mentioned above, this will require a lot of memory - specifically, about 9 times more than the original optimization at a single $k$-point. Here is how this can be overcome at a small cost of computational time.

In [20]:
# We set gmax=1 for this example to avoid memory issues
gmax = 1

# Objective function defining the average imaginary part
def fim_kavg(params):
    (gme, fims) = gme_cavity_k(params, gmax, options, np.vstack((kxg, kyg)))
    # Scale for easier printing
    return npa.mean(fims)*1e6

# Compute the gradient and time the computation; print just the first value
obj_grad = value_and_grad(fim_kavg)
t = time.time()
grad_a = obj_grad(pstart)[1]
print("Autograd gradient:  %1.4f, computed in %1.4fs" %(grad_a[0], time.time() - t))

Autograd gradient:  269.5663, computed in 22.0130s


Below is an altenative way to do the same thing; if you compare the memory usage between the cell below and the one above, you'll realize the purpose of this whole thing.

We use a custom function that mimics `map(lambda f: f(params), fns))` in a way that splits the gradient computation instead of storing all the intermediate variables for all functions. NB: the function `fns` all have to return a scalar and `params` are all vectors. `fmap` then returns an array of the same size as the number of functions in `fns`.

In [21]:
from legume.primitives import fmap

def of_kavg_fmap(params):
    # A function factory to make a list of functions for every k-point
    def fim_factory(ik):
        def fim(params):
            (gme, freq_im) = gme_cavity_k(params, gmax, options, np.array([[kxg[ik]], [kyg[ik]]]))
            return freq_im
        return fim
    
    fims = fmap([fim_factory(ik=ik) for ik in range(nkx*nky)], params)
    return npa.mean(fims)*1e6

# Compute the gradient and time the computation; print just the first value
obj_grad = value_and_grad(of_kavg_fmap)
t = time.time()
grad_a = obj_grad(pstart)[1]
print("Autograd gradient:  %1.4f, computed in %1.4fs" %(grad_a[0], time.time() - t))

Autograd gradient:  269.5663, computed in 26.9777s


In [ ]:
%load_ext memory_profiler